<a href="https://colab.research.google.com/github/jeffheaton/app_generative_ai/blob/main/t81_559_class_03_5_book.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-559: Applications of Generative Artificial Intelligence
**Module 3: Large Language Models**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 3 Material

* Part 3.1: Foundation Models [[Video]](https://www.youtube.com/watch?v=Gb0tk5qq1fA) [[Notebook]](t81_559_class_03_1_llm.ipynb)
* Part 3.2: Text Generation [[Video]](https://www.youtube.com/watch?v=lB97Lqt7q58) [[Notebook]](t81_559_class_03_2_text_gen.ipynb)
* Part 3.3: Text Summarization [[Video]](https://www.youtube.com/watch?v=3MoIUXE2eEU) [[Notebook]](t81_559_class_03_3_text_summary.ipynb)
* Part 3.4: Text Classification [[Video]](https://www.youtube.com/watch?v=2VpOwFIGmA8) [[Notebook]](t81_559_class_03_4_classification.ipynb)
* **Part 3.5: LLM Writes a Book** [[Video]](https://www.youtube.com/watch?v=iU40Rttlb_Q) [[Notebook]](t81_559_class_03_5_book.ipynb)


# Google CoLab Instructions

The following code ensures that Google CoLab is running and maps Google Drive if needed.

In [2]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain langchain_openai langchain_community pypdf pdfkit
    !apt-get install wkhtmltopdf

Note: not using Google CoLab


# 3.5: LLM Writes a Book

This section will demonstrate how to effectively utilize the LLM to write a book. Writing a book is a uniquely complex challenge that requires planning and execution. Unlike simpler tasks, drafting a full-length book is a journey that extends beyond a single session with an LLM due to its extensive length and depth. However, with a systematic approach, you can harness the capabilities of an LLM to create a comprehensive manuscript, empowering you to conquer this challenge.

This section presents a method to craft a book using an LLM, which involves breaking down the book creation process into manageable segments. Our journey begins with a foundational step: selecting a subject for the book. Once a subject is chosen, the process unfolds through a series of iterative interactions with the LLM, each building upon the last to gradually construct the complete book, igniting your passion for writing.

Initially, we prompt the LLM to generate a random title based on the given subject. This title sets the thematic tone for the book and guides the subsequent steps. Following the title, we request the LLM to create a random synopsis. This synopsis acts as a narrative backbone, offering a brief overview of the book’s content and direction.

With a synopsis, the next step is to generate a table of contents. This table outlines the main chapters and sections of the book, providing a structured framework that organizes the content logically and cohesively. It is from this framework that the book starts to take shape.
The next phase involves creating each chapter. We engage in a focused session with the LLM for every chapter outlined in the table of contents. During each session, we provide the LLM with the synopsis and the complete table of contents to ensure consistency and continuity throughout the book. This iterative process allows each chapter to be crafted with attention to detail, ensuring it aligns with the previously established narrative and thematic elements.

By methodically iterating through these steps, we harness the capabilities of an LLM to transform a simple idea into a detailed and well-structured book. This chapter will guide you through each stage, providing practical advice on collaborating with an LLM to achieve more complex tasks.

We begin by accessing a large language model with a temperature of 0.7. We use a higher temperature to encourage creativity.

In [3]:
from langchain.chains.summarize import load_summarize_chain
from langchain import OpenAI, PromptTemplate
from langchain_openai import ChatOpenAI
from IPython.display import display_markdown

MODEL = 'gpt-4o-mini'

llm = ChatOpenAI(
        model=MODEL,
        temperature=0.7,
        n=1
    )


We create a simple utility function to query the LLM with a custom system prompt. The system prompt tells the LLM that it is assisting in creating a book.

In [4]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_openai import ChatOpenAI

def query_llm(prompt):
  messages = [
      SystemMessage(
          content="You are assistant helping to write a book."
      ),
      HumanMessage(
          content=prompt
      ),
  ]

  output = llm.invoke(messages)
  return output.content


## Generate Title, Synopsis and Table of Contents

For this book, we allow the user to specify the subject specified by the SUBJECT variable. We then request the LLM to generate a random title based on this subject. It is essential that the prompt request that the LLM only the; LLMs often like to prefix with text such as "Here is a random title."

In [5]:
SUBJECT = "international spy thriller"

title = query_llm(f"""
Give me a random title for a book on the subject '{SUBJECT}'.
Return only the title, no additional text.
""").strip(" '\"")
print(title)

Shadows of Deceit


Now that we have a title, we can request a random synopsis of the book.

In [6]:
synopsis = query_llm(f"""
Give me a synopsis for a book of the title '{SUBJECT}' for a book on the subject '{SUBJECT}'.
Return only the synopsis, no additional text.
""").strip(" '\"")
print(synopsis)

In a world where trust is a luxury and secrets are currency, former MI6 agent Elara Reed is pulled back into the shadows after a high-profile assassination shatters the delicate balance of power among the world’s intelligence agencies. With a cryptic warning from her estranged mentor and a rogue faction of spies threatening global security, Elara races against time to uncover a conspiracy that spans continents. As she navigates a treacherous web of double agents, high-stakes espionage, and political intrigue, she must rely on her instincts and unearth hidden truths that could change the course of history. Betrayal lurks at every corner, and as alliances shift, Elara must confront her own past to prevent a catastrophic event that could ignite a world war. In a pulse-pounding chase from London to Moscow, she discovers that the most dangerous game is played not just with weapons, but with the secrets of the heart.


Next, we generate the table of contents. For this generation, we provide all previous information. We also request a particular format for the table of contents. You may notice that I ask for the chapter numbers, even though they are an increasing numeric count and could easily be derived. This process works better because the LLM wants to provide the chapter numbers, and attempts to suppress chapter numbers are often inconsistent. It is easier to allow the LLM to generate chapter numbers but control where it generates them so that I can consistently remove them later.

In [7]:
toc = query_llm(f"""
Give me a table of contents for a book of the title '{title}' for a book on
the subject '{SUBJECT}' the book synopsis is '{synopsis}'.
Return the table of contents as a list of chapter titles.
Separate the chapter number and chapter title with a pipe character '|'.
Return only the chapter names, no additional text.
""").strip(" '\"")
print(toc)

1 | Shadows of the Past  
2 | A Fateful Encounter  
3 | The Ashes of Betrayal  
4 | Cryptic Warnings  
5 | The Call to Action  
6 | In the Crosshairs  
7 | A Web of Lies  
8 | The Hidden Enemy  
9 | Race Against Time  
10 | Allies and Adversaries  
11 | The Heart of Darkness  
12 | Secrets of the Heart  
13 | The City of Spies  
14 | Unraveling Threads  
15 | The Brink of War  
16 | Collateral Damage  
17 | The Final Gambit  
18 | Betrayal Revealed  
19 | A Dangerous Game  
20 | Echoes of Deceit  
21 | The Last Stand  
22 | A New Dawn


We must now parse the table of contents and remove the pipes and chapter numbers.

In [8]:
# Split the string into lines
lines = toc.splitlines()

# Extract titles using list comprehension
toc2 = [line.split('|')[1].strip() for line in lines if line]

# Print the list of titles
print(toc2)

['Shadows of the Past', 'A Fateful Encounter', 'The Ashes of Betrayal', 'Cryptic Warnings', 'The Call to Action', 'In the Crosshairs', 'A Web of Lies', 'The Hidden Enemy', 'Race Against Time', 'Allies and Adversaries', 'The Heart of Darkness', 'Secrets of the Heart', 'The City of Spies', 'Unraveling Threads', 'The Brink of War', 'Collateral Damage', 'The Final Gambit', 'Betrayal Revealed', 'A Dangerous Game', 'Echoes of Deceit', 'The Last Stand', 'A New Dawn']


## Generate the Chapters of the Book

Next, we create a function capable of producing the text that makes up a chapter. To ensure that the function has enough context to generate each chapter, we provide the synopsis, the table of contents, and the chapter number. To test this code, we request that it develop a single chapter.

In [9]:
def render_chapter(num, chapter_title, title, subject, synopsis, toc):
  txt = query_llm(f"""
  Write Chapter {num}, titled "{chapter_title}" for a book of the title '{title}' for a book on
  the subject '{subject}' the book synopsis is '{synopsis}' the table of contents is '{toc}'.
  Give me only the chapter text, no chapter heading, no chapter title, number, no additional text.
  """).strip(" '\"")
  return txt

txt = render_chapter(1, toc2[0], title, SUBJECT, synopsis, toc)
print(txt)

Elara Reed stood at the window of her modest apartment in London, the city sprawled beneath her like a living entity, pulsing with life and secrets. The rain drummed against the glass, a steady rhythm that matched the thumping beat of her heart. It had been years since she traded her life as an MI6 agent for the quietude of anonymity, yet the shadows of her past loomed large, threatening to engulf her once more.

The news blared from the television in the corner, a multitude of voices clamoring for attention. "High-profile assassination rocks the intelligence community," the anchor announced, her face a mask of concern. Elara's gaze sharpened as the details unfolded. A prominent diplomat had been killed in a brazen attack, an act of violence that sent ripples through the already fragile alliances of the world’s intelligence agencies. She felt a chill creep up her spine, her instincts honed from years of training whispering that this was no ordinary killing.

Memories flooded back, unin

We can now generate the entire book in Markdown, which allows some formatting. We begin by rendering the title and synopsis, the table of contents, and each chapter.

In [10]:
book = ""

# Render the title and synopsis
book += f"# {title}\n"
book += f"{synopsis}\n"

# Render the toc
book += f"\n## Table of Contents\n\n"
num = 1
for chapter_title in toc2:
  book += f"{num}. {chapter_title}\n"
  num += 1

# Render the book
chapter = 1
for chapter_title in toc2:
  print(f"Rendering chapter {chapter}/{len(toc2)}: {chapter_title}")
  txt = render_chapter(chapter, chapter_title, title, SUBJECT, synopsis, toc)
  book += f"\n\n## Chapter {chapter}: {chapter_title}\n"
  book += f"{txt}\n"
  chapter += 1

Rendering chapter 1/22: Shadows of the Past
Rendering chapter 2/22: A Fateful Encounter
Rendering chapter 3/22: The Ashes of Betrayal
Rendering chapter 4/22: Cryptic Warnings
Rendering chapter 5/22: The Call to Action
Rendering chapter 6/22: In the Crosshairs
Rendering chapter 7/22: A Web of Lies
Rendering chapter 8/22: The Hidden Enemy
Rendering chapter 9/22: Race Against Time
Rendering chapter 10/22: Allies and Adversaries
Rendering chapter 11/22: The Heart of Darkness
Rendering chapter 12/22: Secrets of the Heart
Rendering chapter 13/22: The City of Spies
Rendering chapter 14/22: Unraveling Threads
Rendering chapter 15/22: The Brink of War
Rendering chapter 16/22: Collateral Damage
Rendering chapter 17/22: The Final Gambit
Rendering chapter 18/22: Betrayal Revealed
Rendering chapter 19/22: A Dangerous Game
Rendering chapter 20/22: Echoes of Deceit
Rendering chapter 21/22: The Last Stand
Rendering chapter 22/22: A New Dawn


## Generate a PDF of the Book

Now that we have generated the book, we have saved it as a PDF.

In [14]:
# import markdown
# import pdfkit

# # Convert Markdown to HTML
# html = markdown.markdown(book)
# options = {
#     'encoding': "UTF-8",  # Ensures UTF-8 encoding
# }

# pdfkit.from_string(html, 'output.pdf', options=options)


# import markdown
# from weasyprint import HTML

# # Sample Markdown content (you can replace this with your actual content)
# book = "# Sample Markdown Content\nThis is a sample content."

# # Convert Markdown to HTML
# html_content = markdown.markdown(book)

# # Convert HTML to PDF using WeasyPrint
# HTML(string=html_content).write_pdf('output.pdf')

# print("PDF generated successfully: output.pdf")

We can now download the generated book.

In [11]:
# from google.colab import files
# files.download("output.pdf")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>